# Sandbox For Learning TensorFlow

import tensorflow

In [9]:
import tensorflow as tf

Checking to see if tensorflow is using accelerated hardware

In [15]:
if tf.config.list_physical_devices('GPU'):
  print("TensorFlow **IS** using the GPU")
else:
  print("TensorFlow **IS NOT** using the GPU")

TensorFlow **IS NOT** using the GPU


create a variable: x.

In [11]:
x = tf.constant([[1,2,3],[4,5,6]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)
(2, 3)
<dtype: 'int32'>


In [14]:
print(x + 5)  # adding 5 to each value of x
print(x * 2)  # multiplying each value of x by 2
print(tf.concat([x, x, x], axis=1)) # concatenating the tensors together side by side
print(tf.reduce_sum(x)) # finds the sum of all of the elements in the tensor

tf.Tensor(
[[ 6  7  8]
 [ 9 10 11]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[ 2  4  6]
 [ 8 10 12]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[1 2 3 1 2 3 1 2 3]
 [4 5 6 4 5 6 4 5 6]], shape=(2, 9), dtype=int32)
tf.Tensor(21, shape=(), dtype=int32)


In [19]:
var = tf.Variable([0,0,0]) # creating a variable
var.assign([1,2,3]) # assigning values to the variable
var.assign_add([1,1,1]) # adding values to each element of the tensor and saving it

<tf.Variable 'UnreadVariable' shape=(3,) dtype=int32, numpy=array([2, 3, 4])>

Tensorflow can apparently find the derrivatives of functions. This comes in very useful for things like gradient decent when calculating the loss and error of a model given a set of weights.

In [25]:
x = tf.Variable(1.0)

# we create a random function
def f(x):
    y = x**2 + 2*x - 5
    return y

f(x) # this will return -2 because f(1) = -2

# this will find the derrivative of a function
with tf.GradientTape() as tape:
    y = f(x)

g_x = tape.gradient(y, x)  # g(x) = dy/dx

g_x

<tf.Tensor: shape=(), dtype=float32, numpy=4.0>

Tensorflow also provides tools for optimization and saving your models once they are done training. To do this you are required to using tf.function. When making a tensorflow functions with the @tf.function it is called a graph and can be used to significantly speed up calculations and can also be exports on a system without a python installation.

In [34]:
@tf.function
def my_func(x): # this is our graph
  print('Tracing.\n')
  return tf.reduce_sum(x)

x = tf.constant([1,2,3])

my_func(x) # first call will be unoptimized and still call python methods
my_func(x) # second and subsequent calls will just use TF.

Tracing.

^ Unoptimized Function
Optimized Function


modules are a useful tool in tensorflow because they can allow you to save the values of your variables while training allowing you to load and save them with ease (using tf.train.Checkpoint). Additionally, you can save your variables and models which can allow you to run your model independently from your python code (using tf.saved_model).

In [36]:
class MyModule(tf.Module): # create our module
  def __init__(self, value): # constructor 
    self.weight = tf.Variable(value) 

  @tf.function # create a function
  def multiply(self, x): 
    return x * self.weight # multiply a value by the weight 

testing the module

In [59]:
mod = MyModule(3)
mod.multiply(tf.constant([1, 2, 3]))

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([3, 6, 9])>

saving our module

In [51]:
save_path = './saved'
tf.saved_model.save(mod, save_path) # saving our module to the path "./saved"

INFO:tensorflow:Assets written to: ./saved\assets


reloading our module is as easy as.

In [52]:
reloaded = tf.saved_model.load(save_path)
#reloaded.multiply(tf.Variable([7, 4, 2]))

In [57]:
mod.multiply(tf.Variable([1,2,3]))
reloaded.multiply(2)

ValueError: Could not find matching function to call loaded from the SavedModel. Got:
  Positional arguments (1 total):
    * 2
  Keyword arguments: {}

Expected these arguments to match one of the following 1 option(s):

Option 1:
  Positional arguments (1 total):
    * VariableSpec(shape=(3,), dtype=tf.int32, name='x')
  Keyword arguments: {}